In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 494.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('/content/drive/MyDrive/travel_advisor_Paris_2024/.env')) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']


In [4]:
#from utils import get_serper_api_key
serper_api_key = os.environ["SERPER_API_KEY"]

In [40]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [6]:
from crewai import Agent, Task, Crew

In [51]:
import requests
import json

from langchain.tools import tool
from langchain_community.document_loaders import WebBaseLoader

class SearchTools:

  @tool('search olympic events')
  def search_olympic_events(query: str) -> str:
    """
    Use this tool to search the internet for 2024 Paris Olympics venues and events. This tools returns 5 results from Google search engine.
    """
    return SearchTools.search(f"site:olympics.com/en/news/olympic-games-paris-2024-full-schedule-and-day-by-day-competitions {query}", limit=5)

  @tool('search public transport')
  def search_public_transport(query: str) -> str:
    """
    Use this tool to search the internet for public transportation available near olympic venues, restaurants, tourist attractions. \n
    This tools returns 5 results from Google search engine.
    """
    return SearchTools.search(f"site:ratp.fr/en {query}", limit=5)

  @tool('open page')
  def open_page(url: str) -> str:
    """
    Use this tool to open a webpage and get the content.
    """
    loader = WebBaseLoader(url)
    return loader.load()


  def search(query, limit=5):

    url = "https://google.serper.dev/search"
    payload = json.dumps({
      "q": query,
      "num": limit,
    })
    headers = {
      'X-API-KEY': os.getenv("SERPER_API_KEY"),
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['organic']

    string = []
    for result in results:
      string.append(f"{result['title']}\n{result['snippet']}\n{result['link']}\n\n")

    return f"Search results for '{query}':\n\n" + "\n".join(string)

In [52]:
local_guide = Agent(
    role="Paris tour guide",
    goal="Recommend local attractions including 'hidden gems' and local experiences to a visitor according to"
          "their {interests}, avoid overcrowded tourist areas",
    backstory = "You have an extensive knowledge of Paris attractions and can give visitors"
                "detailed recommendations of places to visit taking their {interests} into consideration. Your recommendations always"
                "include lesser known attractions ('hidden gems') that allow the visitors"
                "to avoid crowded toursit areas and experience the authentic side of Paris.",
    allow_delegation=False,
	  verbose=True
)

In [53]:
olympic_guide = Agent(
    role="An expert on Olympic games schedule",
    goal="Recommend Olympic events to a visitor according to their {interests} and {travel_dates}",
    backstory="You have an extensive knowledge of the Olympic games schedules. You can recommend the most"
              "exciting Olympic events to a visitor to Paris taking their {interests} and {travel_dates} into consideration."
              "You only recommend actual 2024 Olympic events. If you cannot recommend any, just say"
              "there are no events (do not make any up).",
    allow_delegation=False,
    verbose=True)

In [54]:
public_transport_guide = Agent(
    role="An expert on public transportation in Paris",
    goal="Give a recommendation on how to get to events recommended by the Olympic games expert and places recommended"
          "by the Paris tour guide using public transportation ",
    backstory="You have an extensive knowledge of public transportation and all the routes in Paris. You"
              "can always give a recommendation on the best way to get to a place using public transportation.",
    allow_delegation=False,
    verbose=True
)

In [55]:
travel_planner = Agent(
        role="Amazing travel planner",
        goal="Create the most amazing travel itineraries for travellers to Paris for {days} days including Olympic events"
              "if applicable and public transportation recommendations",
        backstory="You are a specialist in travel planning with decades of experience."
                   "You provide a detailed travel itinerary for number of {days} days,"
                   "that includes the recommendations of the Paris tour guide, the expert on Olympic schedule of events"
                   "and the expert on public transport. ",
        verbose=True)

In [56]:
recommend = Task(
    description=(
        "1. Recommend local attractions for a visitor according to their"
            "{interests}. Prioritize lesser known attractions that are away from tourist crowds.\n"
        "2. Recommend authentic Parisian experiences according to "
            "visitor's {interests}.\n"
        "3. Recommend authentic local restaurants.\n "
    ),
    expected_output="A list of items, where each item includes the name of a place in Paris, location of the place,"
                    "and a short description of why this place is worth visiting",
    agent=local_guide,
)

In [57]:
find_olympic_events = Task(
    description="Find Olympic events according to visitor's {interests} that happen during {travel_dates}",
    expected_output="A list of Olympic events with location, date and time.",
    agent=olympic_guide,
    tools=[SearchTools.search_olympic_events, SearchTools.open_page]
)

In [58]:
find_public_transport = Task(
    description=(
        "1. Recommend how to get from a given place using"
            "public transportation. Recommend the fastest route."
        "2. Recommend an alternative route with fewest transfers."
    ),
    expected_output="A specific list of modes of transportation and route numbers to get to a place",
    agent=public_transport_guide,
    tools=[SearchTools.search_public_transport, SearchTools.open_page]
)

In [59]:
plan = Task(
    description = (
        "1. Expand the recommendations of the Paris tour guide into a a full {days}-day travel"
            "itinerary with detailed per-day plans, including places to eat.\n"
        "2. This itinerary should cover all aspects of the trip, from arrival to departure,"
            "integrating the Paris tour guide information with practical travel logistics."
        "3. You MUST suggest actual places to visit and actual restaurants to go to."
        "4. Recommend 2024 Paris Olympic events according to {interests} if they fall within {travel_dates}"
        "5. You MUST give public transportation nearby the attractions, restaurants and Olympic events that you recommended"
    ),
    expected_output = "Your final answer MUST be a complete expanded travel plan,"
                      "formatted as markdown, encompassing a daily schedule,"
                      "ensuring THE BEST TRIP EVER. Be specific and give a reason why you picked"
                      "each place, what make them special! Don't forget to mention public transportation near each place you picked",
    agent=travel_planner,
    context=[recommend, find_olympic_events, find_public_transport]
)

In [60]:
crew = Crew(
    agents=[local_guide, olympic_guide, public_transport_guide, travel_planner],
    tasks=[recommend, find_olympic_events, find_public_transport, plan],
    verbose=2
)

In [61]:
result = crew.kickoff(inputs={"interests": "Art, tennis", "days": 4, "travel_dates": "August 1 - August 5, 2024"})

 [DEBUG]: == Working Agent: Paris tour guide
 [INFO]: == Starting Task: 1. Recommend local attractions for a visitor according to theirArt, tennis. Prioritize lesser known attractions that are away from tourist crowds.
2. Recommend authentic Parisian experiences according to visitor's Art, tennis.
3. Recommend authentic local restaurants.
 


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

1. Local Attractions for Art and Tennis Enthusiasts:
- Palais de Tokyo (13 Avenue du Président Wilson, 75116 Paris): This contemporary art museum showcases cutting-edge exhibitions and installations, perfect for art lovers.
- Tenniseum at Roland Garros (2 Avenue Gordon Bennett, 75016 Paris): This museum is dedicated to the history of tennis and the prestigious Roland Garros tournament, a must-visit for tennis enthusiasts.
- Le Centquatre (5 Rue Curial, 75019 Paris): A vibrant cultural center with art exhibitions, workshops, and performances, offering a unique 